In [ ]:
# Required Packages to install
# ! pip install cmake --user
#! pip install dlib --user
# ! pip install moviepy --user
# ! pip install ffmpeg --user

In [ ]:
# First, set up the required folders

# os.mkdir('raw_images')
# os.mkdir('aligned_images')
# os.mkdir('generated_images')

In [8]:
import ffmpeg
import imageio
import os
import random
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
URL_FFHQ = 'https://s3-us-west-2.amazonaws.com/nanonets/blogs/karras2019stylegan-ffhq-1024x1024.pkl'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs = pickle.load(f)

generator = Generator(Gs, batch_size=1, randomize_noise=False)

W1110 13:34:41.488071 139987547989824 deprecation_wrapper.py:119] From /home/rbez/ECE199/stylegan-encoder/dnnlib/tflib/tfutil.py:97: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1110 13:34:41.489997 139987547989824 deprecation_wrapper.py:119] From /home/rbez/ECE199/stylegan-encoder/dnnlib/tflib/tfutil.py:109: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W1110 13:34:46.002216 139987547989824 deprecation.py:323] From <string>:364: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
def generate_image_for_video(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    
    return img_array

In [52]:
# 1) Extract and align faces from images
!python align_images.py raw_images/ aligned_images/

Using TensorFlow backend.
Traceback (most recent call last):
  File "align_images.py", line 6, in <module>
    from ffhq_dataset.landmarks_detector import LandmarksDetector
  File "/home/rbez/ECE199/stylegan-encoder/ffhq_dataset/landmarks_detector.py", line 1, in <module>
    import dlib
ModuleNotFoundError: No module named 'dlib'


In [ ]:
# 2) Find latent representation of aligned image
#    This cell may take a long time (~1 hr)
!python encode_images.py aligned_images/ generated_images/ latent_representations/

In [21]:
# Loading leanred latent representations

jon = np.load('latent_representations/jon.npy')
drogo = np.load('latent_representations/drogo.npy')
bran = np.load('latent_representations/bran.npy')
cersei = np.load('latent_representations/cersei.npy')
daenerys = np.load('latent_representations/daenerys.npy')

In [28]:
# If you're interpolating between two characters, set these characters here
char1 = jon
char2 = drogo

# This creates an nd array that stores all the image frames fot cross-character interpolation
src_images = np.stack(generate_image_for_video((0.01*alpha*char2)+((1-(0.01*alpha))*char1)) for alpha in range (100))

imageio.mimwrite('test2char.mp4',src_images, fps = 20)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


In [44]:
# Attempt at interpolating between many characters
chars = [jon, drogo, cersei]
random_walk = [i for i in range(len(chars))]
random.shuffle(random_walk)
frames = []
i = 0

while i < len(random_walk)-1:
    frames.extend(np.stack(generate_image_for_video((0.01*alpha*chars[random_walk[i+1]])+((1-(0.01*alpha))*chars[random_walk[i]])) for alpha in range (100)))
    i+=1

#print(frames)
imageio.mimwrite('test3chars120.mp4',np.array(frames), fps = 120)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if __name__ == '__main__':


In [43]:
# # Attempts at modifying duration, smoothing, fps, num_frames

# duration_sec = 10.0
# smoothing_sec = 2.0
# mp4_fps = 20
# num_frames = int(np.rint(duration_sec * mp4_fps))

# def make_frame(t):
#     frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
#     src_image = src_images[frame_idx]
#     return np.array(src_image)


# chars = [jon, drogo, cersei]
# random_walk = [0,1,2]
# random.shuffle(random_walk)
# frames = []
# i = 0

# while i < len(random_walk)-1:
#     frames.extend(np.stack(generate_image_for_video((0.01*alpha*chars[random_walk[i+1]])+((1-(0.01*alpha))*chars[random_walk[i]])) for alpha in range (100)))
#     i+=1
    
# made_frames = []

# for each in frames:
#     im = imageio.imread(each)
#     made_frames.extend(make_frame(im))    

# #print(frames)
# imageio.mimwrite('test_duration.mp4',np.array(made_frames), fps = 20)

In [51]:
# Testing a looping interpolation
chars = [jon, drogo, cersei]
random_walk = [i for i in range(len(chars))]
random.shuffle(random_walk)
frames = []
i = 0

while i < len(random_walk)-1:
    frames.extend(np.stack(generate_image_for_video((0.01*alpha*chars[random_walk[i+1]])+((1-(0.01*alpha))*chars[random_walk[i]])) for alpha in range (100)))
    i+=1
    
frames.extend(np.stack(generate_image_for_video((0.01*alpha*chars[random_walk[i+1]])+((1-(0.01*alpha))*chars[random_walk[0]])) for alpha in range (100)))

#print(frames)
imageio.mimwrite('testlooping.mp4',np.array(frames), fps = 120)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if sys.path[0] == '':


IndexError: list index out of range